# Setup

In [1]:
%load_ext autoreload
%autoreload 2 
#2

In [ ]:
# ensure dependencies are installed
#!pip install polars
#!pip install pandas
#!pip install numpy
#!pip install matplotlib
#!pip install flax
#!pip install scikit-image
#!pip install git+https://github.com/wcarvalho/nicewebrl
#!pip install git+https://github.com/wcarvalho/JaxHouseMaze.git
#!pip install --upgrade git+https://github.com/wcarvalho/JaxHouseMaze.git
#!pip install --upgrade --force --no-cache-dir git+https://github.com/wcarvalho/nicewebrl
#!pip install --upgrade jax jaxlib --platform osx-arm64
#!pip uninstall jax jaxlib
#!pip install --upgrade "jax[cpu]" jaxlib --platform osx-arm64

Found existing installation: jax 0.4.33
Uninstalling jax-0.4.33:
  Would remove:
    /Users/hall-mcmaster/miniconda3/envs/human-dyna-web-arm64/lib/python3.10/site-packages/jax-0.4.33.dist-info/*
    /Users/hall-mcmaster/miniconda3/envs/human-dyna-web-arm64/lib/python3.10/site-packages/jax/*
Proceed (Y/n)? 

In [2]:
# import packages
import os
from os.path import join as opj
import sys

# add parent directory to current path
cdir=os.getcwd()
parent_dir = os.path.dirname(cdir)
sys.path.append(parent_dir)
print(parent_dir)

from typing import List
import numpy as np

import json
import copy
import load_data
from pprint import pprint
import matplotlib.pyplot as plt
import polars as pl
import pandas as pd
import experiment_1 as experiment

from collections import defaultdict
import jax.numpy as jnp
import jax.tree_util as jtu
import jax

from load_data import get_timestep, compute_reaction_time, EpisodeData
from load_data import render_episode, get_task_object, object_idx_to_name, success_fn, get_task_room, create_maps
from load_data import filter_episodes


/Users/hall-mcmaster/Documents/Projects/Maze/human-dyna-web
No file specified for image dict.
Using: /Users/hall-mcmaster/miniconda3/envs/human-dyna-web-arm64/lib/python3.10/site-packages/housemaze/image_data.pkl
Compiling jax environment functions.
	time: 0.4362449645996094
Compiling multi-render function.
	time: 0.06319332122802734


# Define functions

In [3]:
def separate_data_by_block_stage(data: List[dict]):
    """This function will group episodes by the values from get_block_stage_description

    The input i
    So for example, each episode with {'stage': "'not obvious' shortcut",
     'block': 'shortcut',
     'manipulation': 1,
     'episode_idx': 1,
     'eval': True}
     with go into its own list.
    """
    grouped_data = defaultdict(list)
    episode_idx = -1
    keys = set()
    infos = dict()
    # first group all of the data based on which (stage, block) its in
    for datum in data:
        info = load_data.get_block_stage_description(datum)
        key = load_data.dict_to_string(info)
        if not key in keys:
            episode_idx += 1
            keys.add(key)
        info['global_episode_idx'] = episode_idx
        
        updated_key = load_data.dict_to_string(info)
        grouped_data[updated_key].append(datum)
        infos[updated_key] = info
    return grouped_data, infos


In [4]:
def went_to_junction(positions, junction):
    match = np.array(junction) == positions
    match = (match).sum(-1) == 2 # both x and y matches
    return match.any()  # if any matched

def followed_junction(episode_data, junction_pos=(0,11)):
    return went_to_junction(episode_data.positions, junction_pos)



In [6]:
###########
# manual
###########
eval_df = user_df.filter(manipulation=1, eval=True)
idxs = np.array(eval_df['global_episode_idx'])-1
array = []
for idx in idxs:
    val = followed_junction(user_data[idx])
    array.append(val)

###########
# automatic
###########
def get_human_data(user_df, user_data, fn, **kwargs):
    eval_df = user_df.filter(**kwargs)
    idxs = np.array(eval_df['index'])-1
    array = []
    for idx in idxs:
        val = went_to_junction(user_data[idx])
        array.append(val)

    return array

array = get_human_data(user_df, user_data, fn=followed_junction, manipulation=1, eval=True)

NameError: name 'user_df' is not defined

In [7]:
file = file_list[0]
with open(file, 'r') as f:
    data_dicts = json.load(f)
if exp_string=='pilot-1':
    data_dicts = [row for row in data_dicts if not 'practice' in row['metadata']['block_metadata']['desc']]
else:
    data_dicts = [row for row in data_dicts[:-1] if not 'practice' in row['metadata']['block_metadata']['desc']]

NameError: name 'file_list' is not defined

In [8]:
from housemaze import utils
from housemaze import renderer

image_data = utils.load_image_dict()

def render_path(timestep, actions):
    # render the image from t=0
    def fn(x): return x[0]
    t0 = jax.tree_map(lambda x: x[0], timestep) 
    image = renderer.create_image_from_grid(
        t0.state.grid,
        t0.state.agent_pos,
        t0.state.agent_dir,
        image_data)

    # get path and actions from all timesteps
    path = timestep.state.agent_pos[:-1]
    print(f"Path length: {len(path)}")
    renderer.place_arrows_on_image(image, path, actions, *timestep.state.grid[0].shape[:2])
    plt.show()

gds, gd_infos = separate_data_by_block_stage(data_dicts)

episode_data = [None]*len(gds.keys())
episode_info = [None]*len(gds.keys())
for key in gds.keys():
    red = raw_episode_data = gds[key]
    
    # get actions
    actions = jnp.asarray([datum['action_idx'] for datum in red])

    # collect timesteps
    timesteps = [get_timestep(datum) for datum in red]
    
    # combine them into trajectory
    timesteps = jtu.tree_map(
            lambda *v: jnp.stack(v), *timesteps)
    render_path(timesteps, actions)
    break

No file specified for image dict.
Using: /Users/hall-mcmaster/miniconda3/envs/human-dyna-web-arm64/lib/python3.10/site-packages/housemaze/image_data.pkl


NameError: name 'data_dicts' is not defined

In [13]:
def make_episode_data(data: List[dict], sub_ID=100):
    """This groups all of the data by block/stage information and prepares 
        (1) a list of EpisodeData objects per block/stage
        (2) a dataframe which summarizes all episode information.

    The dataframe can be used to get indices into the list of EpisodeData for further computation.
    """
    gds, gd_infos = separate_data_by_block_stage(data)

    episode_data = [None]*len(gds.keys())
    episode_info = [None]*len(gds.keys())
    for key in gds.keys():
        red = raw_episode_data = gds[key]
        
        # get actions
        actions = jnp.asarray([datum['action_idx'] for datum in red])

        # collect timesteps
        timesteps = [get_timestep(datum) for datum in red]
        
        # combine them into trajectory
        timesteps = jtu.tree_map(
                lambda *v: jnp.stack(v), *timesteps)
        
        positions = timesteps.state.agent_pos

        reaction_times = [compute_reaction_time(datum) for datum in red]
        reaction_times = jnp.asarray(reaction_times)

        episode_idx = gd_infos[key]['global_episode_idx']
        episode_data[episode_idx] = EpisodeData(
            actions=actions,
            positions=positions,
            reaction_times=reaction_times,
            timesteps=timesteps,
            
        )

        ###############################################################
        # Select variables for episode level dataframe
        ###############################################################
        datum0 = red[0] 
        # add user ID, age and sex
        info = datum0['user_data']
        # replace ID from the raw data with an anonymised subject value
        info['user_id']=sub_ID
        info.update(copy.deepcopy(gd_infos[key]))
        info.update(
            goal_object_numeric=get_task_object(timesteps),
            goal_object_string=object_idx_to_name(get_task_object(timesteps)),
            goal_object_reached=success_fn(episode_data[episode_idx].timesteps),
            actions=actions.tolist(),
            reaction_times=reaction_times.tolist(),
            room=get_task_room(timesteps),
            positions=positions.tolist(),
        )
   
        # add whether participant followed the 'old path' or 'new path'
        # note that positions are based on experiment 2 data
        current_manipulation=info['manipulation']
        evaluation_trl=info['eval']
        #print(evaluation_trl)
        if evaluation_trl==1 and current_manipulation!=2: # manipulation 2 is an RT manipulation
            print('current_manipulation:' + str(current_manipulation))
            print('goal reached:' + str(info['goal_object_reached']))
            if current_manipulation==1:  
                #old_junction=(2,18)
                #new_junction=(15,26)
                old_junction=(2,14)
                new_junction=(14,25)
            #elif current_manipulation==2: # manipulation 2 is an RT manipulation
            #    old_junction=(15,26)
            #    new_junction=(2,3)
            elif current_manipulation==3:
                #old_junction=(8,26)
                #old_junction=(3,26)
                #new_junction=(0,23)
                old_junction=(17,17)
                new_junction=(0,20)
                
            crossed_old = followed_junction(episode_data[episode_idx],old_junction)
            crossed_new = followed_junction(episode_data[episode_idx],new_junction)
            print('crossed_old:' + str(crossed_old))
            print('crossed_new:' + str(crossed_new))

            if info['goal_object_reached'] and not (crossed_old or crossed_new):
                render_path(timesteps, actions)

            info.update(
                crossed_old_path=crossed_old,
                crossed_new_path=crossed_new,
            )
        else:
           info.update(
                crossed_old_path='n/a',
                crossed_new_path='n/a',
            )
            
        # add?
        #'timesteps', pretty complex variable, any worth having from there at the episode level
        
        episode_info[episode_idx] = info

    episode_info = pd.DataFrame(episode_info)
    return episode_data, episode_info

In [14]:
# locate relevant files
def list_files(directory):
    fnames=[]
    for root, dirs, files in os.walk(directory):
        for file in files:
            cfile=opj(root,file)
            fnames.append(cfile)
    return fnames

# Preprocess files

In [15]:
# establish input directory
cdir=os.getcwd()
project_dir=os.path.dirname(cdir)
maze_dir=os.path.dirname(project_dir)
#exp_string='pilot-1-subset'
exp_string='pilot-2-subset'
data_dir=opj(maze_dir,'human-dyna-web-raw-data', exp_string)
file_list=list_files(data_dir)
#print(file_list)

# establish input directory
#exp_string_out='pilot-1'
exp_string_out='pilot-2'
output_dir=opj(project_dir,'data', exp_string_out)
os.makedirs(output_dir, exist_ok=True)

# open each file in the list
sub_count=0
for file in file_list:
    with open(file, 'r') as f:
        data_dicts = json.load(f)

        # remove all blocks that are "practice"
        if exp_string=='pilot-1':
            data_dicts = [row for row in data_dicts if not 'practice' in row['metadata']['block_metadata']['desc']]
        else:
            data_dicts = [row for row in data_dicts[:-1] if not 'practice' in row['metadata']['block_metadata']['desc']]
            #data_dicts = [row for row in data_dicts[:-1] if 'metadata' in row and 'block_metadata' in row['metadata'] and 'desc' in row['metadata']['block_metadata'] and 'practice'

    
        # extract data
        # NOTE: manipulation room is __always__ room 0, even if task objects differ
        sub_count+=1
        print('Extracting data from sub' + str(sub_count))
        all_episode_data, all_episode_info = make_episode_data(data_dicts, sub_ID=sub_count)

        break


Extracting data from sub1


KeyboardInterrupt: 

In [16]:
# establish input directory
cdir=os.getcwd()
project_dir=os.path.dirname(cdir)
maze_dir=os.path.dirname(project_dir)
#exp_string='pilot-1-subset'
exp_string='pilot-2-subset'
data_dir=opj(maze_dir,'human-dyna-web-raw-data', exp_string)
file_list=list_files(data_dir)
#print(file_list)

# establish input directory
#exp_string_out='pilot-1'
exp_string_out='pilot-2'
output_dir=opj(project_dir,'data', exp_string_out)
os.makedirs(output_dir, exist_ok=True)

# open each file in the list
sub_count=0
for file in file_list:

    # if output file has already been produced skip
    filename = 'sub-' + str(sub_count+1).zfill(2) + '.csv'
    full_path = opj(output_dir, filename)
    if os.path.exists(full_path):
        sub_count+=1
        continue
    
    with open(file, 'r') as f:
        data_dicts = json.load(f)

        # remove all blocks that are "practice"
        if exp_string=='pilot-1':
            data_dicts = [row for row in data_dicts if not 'practice' in row['metadata']['block_metadata']['desc']]
        else:
            data_dicts = [row for row in data_dicts[:-1] if not 'practice' in row['metadata']['block_metadata']['desc']]
            #data_dicts = [row for row in data_dicts[:-1] if 'metadata' in row and 'block_metadata' in row['metadata'] and 'desc' in row['metadata']['block_metadata'] and 'practice'

    
        # extract data
        # NOTE: manipulation room is __always__ room 0, even if task objects differ
        sub_count+=1
        print('Extracting data from sub' + str(sub_count))
        all_episode_data, all_episode_info = make_episode_data(data_dicts, sub_ID=sub_count)

        # minor changes to dataframe order and naming
        all_episode_info = all_episode_info.rename(columns={"user_id": "ID"})
        column_order = [
            "ID", 
            "age", 
            "sex", 
            "block", 
            "manipulation", 
            "stage", 
            "eval", 
            "episode_idx", 
            "global_episode_idx", 
            "goal_object_string", 
            "goal_object_numeric",
            "goal_object_reached",
            "crossed_old_path",
            "crossed_new_path",
            "actions",
            "reaction_times",
            "room",
            "positions"
            
        ]
        all_episode_info = all_episode_info[column_order]

        # save output file
        filename='sub-' + str(sub_count).zfill(2) + '.csv'
        full_path = opj(output_dir, filename)
        all_episode_info.to_csv(full_path, index=False)

Extracting data from sub6
current_manipulation:3
goal reached:True
crossed_old:True
crossed_new:False
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True
Extracting data from sub8
current_manipulation:3
goal reached:False
crossed_old:True
crossed_new:False
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True
Extracting data from sub12
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True
current_manipulation:3
goal reached:True
crossed_old:False
crossed_new:True
Extracting data from sub14
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True
current_manipulation:3
goal reached:False
crossed_old:False
crossed_new:False
Extracting data from sub23
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True
current_manipulation:3
goal reached:False
crossed_old:True
crossed_new:False
Extracting data from sub26
current_manipulation:1
goal reached:True
crossed_old:False
crossed_new:True


In [107]:
all_episode_info.head()

,user_id,age,sex,stage,block,manipulation,episode_idx,eval,global_episode_idx,goal_object_numeric,goal_object_string,goal_object_reached,actions,reaction_times,room,positions,crossed_old_path,crossed_new_path
0,1,34,Male,training,shortcut,1,1,False,0,3,spoon,True,"[2, 1, 1, 0, 0, 1, 1, 2, 1, 1, 1, 0, 0, 1, 1, ...","[940.0, 194.0, 152.0, 199.0, 228.0, 186.0, 168...",1,"[[6, 5], [6, 4], [7, 4], [8, 4], [8, 5], [8, 6...",n/a,n/a
1,1,34,Male,training,shortcut,1,2,False,1,43,knife,True,"[1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...","[960.0, 166.0, 127.0, 117.0, 187.0, 141.0, 116...",1,"[[2, 20], [3, 20], [4, 20], [4, 20], [4, 21], ...",n/a,n/a
2,1,34,Male,training,shortcut,1,3,False,2,43,knife,True,"[0, 3, 3, 3, 3, 2, 2, 1, 1, 2, 2, 2, 1]","[861.0, 191.0, 167.0, 156.0, 138.0, 153.0, 157...",1,"[[8, 15], [8, 16], [7, 16], [6, 16], [5, 16], ...",n/a,n/a
3,1,34,Male,training,shortcut,1,4,False,3,3,spoon,True,"[1, 1, 0, 0, 0, 3, 3, 2, 1]","[740.0, 139.0, 164.0, 188.0, 143.0, 142.0, 147...",1,"[[2, 20], [3, 20], [4, 20], [4, 21], [4, 22], ...",n/a,n/a
4,1,34,Male,training,shortcut,1,5,False,4,3,spoon,True,"[3, 3, 3, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[789.0, 113.0, 160.0, 167.0, 259.0, 219.0, 234...",1,"[[6, 9], [5, 9], [4, 9], [3, 9], [3, 10], [3, ...",n/a,n/a
